In [1]:
import pandas as pd
import sqlalchemy
import math
import os, sys, copy, random, pickle, cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append("../")

%load_ext autoreload
%autoreload 2

from modules.ControlTracking import TrackFactory
from modules.ControlDisplay import Display
from modules.Interproc import DBInterface, GuiviewState

from modules.AnalysisHelpers import ( 
         exploreImgs, buildImgComparisonData, subprocEval)

from modules.EvalHelpers import EvalTracker, EvalDataset
from modules.ControlEval import EvalFactory
from modules.DataSchemas import ScoreSchema
from modules.EvalHelpers import OutcomeData, DFHelper

from modules.EvalHelpers import AggEval, DFHelper

from modules.EvalSuite import EvalSuite

### Build OutcomeData from eval to work with

In [319]:
#different trackers here
b_log, b_eval_log = False, False
vidfn = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'
outcome0 = subprocEval(f_pathfn = vidfn, algo_enum=0, b_log=b_log, b_eval_log=b_eval_log)
outcome1 = subprocEval(f_pathfn = vidfn, algo_enum=1, b_log=b_log, b_eval_log=b_eval_log)
outcome2 = subprocEval(f_pathfn = vidfn, algo_enum=2, b_log=b_log, b_eval_log=b_eval_log)

In [2]:
b_log, b_eval_log = True, True
vidfn = 'data/proc/tmp/dec14/output6.proc1.avi'

algo_enums = [str(x) for x in range(3)]

d_outcome = {}
for _enum in algo_enums:
    
    _outcome = subprocEval(f_pathfn = vidfn, algo_enum=_enum, 
                           b_log=b_log, b_eval_log=b_eval_log)
    
    d_outcome[_enum] = _outcome.copy()


START - running eval module...
[                            ]
------------------------------
output db: data/usr/eval_tmp.db
output tbl: outcome_dataframe
eval time: 10.61
outcome rows: 383 cols: 48 

num frames:                  383 
obj enums scored/tracked:    0,1 / 0 
num scored frames:           10 | 0,22,40...125,188,380 
-------
agg_calcBaselineBallUnitsAway   0.18
agg_checkBothContainsOther      0.88
agg_checkTrackSuccess           0.86
fagg_less_than_20_pix_balls_away   0.28
fagg_less_than_30_pix_success      1.00
-----
subproc time: 12.46

START - running eval module...
[                            ]
------------------------------
output db: data/usr/eval_tmp.db
output tbl: outcome_dataframe
eval time: 10.64
outcome rows: 383 cols: 48 

num frames:                  383 
obj enums scored/tracked:    0,1 / 0 
num scored frames:           10 | 0,22,40...125,188,380 
-------
agg_calcBaselineBallUnitsAway   0.18
agg_checkBothContainsOther      0.88
agg_checkTrackSuccess          

### Show Reports from CmpAlgoReport

In [67]:
from modules.EvalSuite import EvalSuite, CmpAlgoReport

In [68]:
cmpB = CmpAlgoReport(
    benchmark_outcome_data=d_outcome['0'],
    current_outcome_data=d_outcome['2']
             )

In [69]:
cmpB.report1()

,improvements,deprovements,sames
checkTrackSuccess,24,0,359


,improvements,deprovements,sames
calcBaselineBallUnitsAway,1,1,8
checkBothContainsOther,0,0,10
checkTrackSuccess,0,0,10
distanceFromBaseline,1,1,8


,diff,current,benchmark
,mean,mean,mean
agg_checkTrackSuccess,0.063,0.924,0.862
agg_calcBaselineBallUnitsAway,-0.004,-0.184,-0.180
agg_checkBothContainsOther,0.000,1.000,1.000
agg_distanceFromBaseline,-0.043,-9.015,-8.972


diff         current          benchmark  \
                                    mean       n    mean        n      mean   
fagg_less_than_20_pix_balls_away  -0.019   0.000  -0.301    2.000    -0.282   
fagg_less_than_30_pix_success      0.000  24.000   1.000  321.000     1.000   

                                           
                                        n  
fagg_less_than_20_pix_balls_away    2.000  
fagg_less_than_30_pix_success     297.000

In [57]:
mini.T

,fagg_less_than_20_pix_balls_away,fagg_less_than_30_pix_success
mean,0.282109,1.0
n,2.000000,297.0


### Pandas ScratchPad

##### Explore the mini-df

In [56]:
mini = cmpB.benchmark.aggFilterDfh.df
mini

,mean,n
fagg_less_than_20_pix_balls_away,0.282109,2.0
fagg_less_than_30_pix_success,1.000000,297.0


In [74]:
mini_stack = mini.stack()
mini_stack

fagg_less_than_20_pix_balls_away  mean      0.533238
                                  n        16.000000
fagg_less_than_30_pix_success     mean      1.000000
                                  n       310.000000
dtype: float64

In [75]:
mini.index

Index([u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], dtype='object')

In [76]:
mini_stack.index

MultiIndex(levels=[[u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], [u'mean', u'n']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [77]:
mini_stack.T

fagg_less_than_20_pix_balls_away  mean      0.533238
                                  n        16.000000
fagg_less_than_30_pix_success     mean      1.000000
                                  n       310.000000
dtype: float64

##### Easiest Way to combine two dataframes and note them as distinct

In [79]:
#on axis=1, dif-df's are ordered by col, so columns are MutliIndex
axis = 1

d_tmp = {'current':mini, 'benchmark':mini}

df_concat = pd.concat(d_tmp.values(), keys=d_tmp.keys(), axis=axis)

print 'cols:  %s', str(df_concat.columns)
print 'index: %s', str(df_concat.index)

df_concat

cols:  %s MultiIndex(levels=[[u'current', u'benchmark'], [u'mean', u'n']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])
index: %s Index([u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], dtype='object')


current        benchmark       
                                      mean      n      mean      n
fagg_less_than_20_pix_balls_away  0.533238   16.0  0.533238   16.0
fagg_less_than_30_pix_success     1.000000  310.0  1.000000  310.0

In [80]:
# on axis=0, df's are ordered by row, so rows are MultiIndex
axis = 0

d_tmp = {'current':mini, 'benchmark':mini}

df_concat = pd.concat(d_tmp.values(), keys=d_tmp.keys(), axis=axis)

print 'cols:  %s', str(df_concat.columns)
print 'index: %s', str(df_concat.index)

df_concat

cols:  %s Index([u'mean', u'n'], dtype='object')
index: %s MultiIndex(levels=[[u'current', u'benchmark'], [u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])


mean      n
current   fagg_less_than_20_pix_balls_away  0.533238   16.0
          fagg_less_than_30_pix_success     1.000000  310.0
benchmark fagg_less_than_20_pix_balls_away  0.533238   16.0
          fagg_less_than_30_pix_success     1.000000  310.0

##### here's an attempt to combine dataframes with manually created MultiIndex

In [ ]:
iters = [list(mini.index), ['first']]
multi_ind_1 = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
mini1 = pd.DataFrame(mini, index=multi_ind_1)
mini1

In [69]:
iters = [list(mini.index), ['second']]
multi_ind_2 = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
mini2 = pd.DataFrame(data=mini.values, index=multi_ind_2, columns=mini.columns)

mini2

,,mean,n
criteria,dfnum,,
fagg_less_than_20_pix_balls_away,second,0.533238,16.0
fagg_less_than_30_pix_success,second,1.000000,310.0


In [ ]:
iters = [list(mini.index), ['first','second']]
multi_ind = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
multi_ind